<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Quantitative_analysis_Tracefinder_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import pandas as pd

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
Auto_1_pm_df = pd.read_csv('/content/drive/My Drive/Qualitative_Tracefinder_Data/Car_samples_27_11_2023_Auto1_PF_120k_125.csv')
Auto_1_pg_df = pd.read_csv('/content/drive/My Drive/Qualitative_Tracefinder_Data/Car_samples_27_11_2023_Auto1_PUF_hc_120k_138.csv')
Auto_2_gp_df = pd.read_csv('/content/drive/My Drive/Qualitative_Tracefinder_Data/Car_samples_27_11_2023_Auto2_PF_120k_147.csv')
Auto_2_pm_df = pd.read_csv('/content/drive/My Drive/Qualitative_Tracefinder_Data/Car_samples_27_11_2023_Auto2_PUF_120k_149.csv')
Auto_3_gp_df = pd.read_csv('/content/drive/My Drive/Qualitative_Tracefinder_Data/Car_samples_27_11_2023_Auto3_PUF_120k_165.csv')